##### Audio Classification Using Edge Impulse
This example uses audio input from the float_wave_test_raw.wav file and uses local Edge Impulse Classification to classigy 5 seconds of audio.

This is based on the Edge Immpulse example:
https://github.com/edgeimpulse/linux-sdk-python/blob/master/examples/custom/classify.py

The following are required to use the edge_impulse_linux:  
<code>
apt update
apt upgrade
apt install libatlas-base-dev libportaudio2 libportaudiocpp0 portaudio19-dev python3-opencv
pip3 install PyAudio
pip3 install edge_impulse_linux</code>

In [1]:
import os
import sys, getopt
import signal
import numpy as np
from edge_impulse_linux.runner import ImpulseRunner
from scipy.io import wavfile

In [2]:
runner = None
model_file_name = 'modelfile.eim'
audio_file_name = 'float_wave_test_raw.wav'

dir_path = os.path.dirname(os.path.realpath(model_file_name))
modelfile = os.path.join(dir_path, model_file_name)
audio_file = os.path.join(dir_path, audio_file_name)

In [3]:
samplerate, generator = wavfile.read(audio_file)

features = np.array([], dtype=np.int16)
runner = ImpulseRunner(modelfile)

model_info = runner.init()
labels = model_info['model_parameters']['labels']
window_size = model_info['model_parameters']['input_features_count']
sampling_rate = model_info['model_parameters']['frequency']
print('Loaded runner for "' + model_info['project']['owner'] + ' / ' + model_info['project']['name'] + '"')
print('labels: ', labels)
print('window_size: ', window_size)
print('sampling_rate: ', sampling_rate)

Loaded runner for "John Bracegirdle / Sound Test"
labels:  ['background-noise', 'edge-impulse', 'red-pitaya']
window_size:  15000
sampling_rate:  3000


In [30]:
samplerate, generator = wavfile.read(audio_file)
features = np.concatenate((features, generator), axis=0)
res = runner.classify(features[:window_size].tolist())
res

{'result': {'classification': {'background-noise': 0.9999896287918091,
   'edge-impulse': 6.324415153358132e-06,
   'red-pitaya': 4.0374552554567344e-06}},
 'timing': {'anomaly': 0,
  'classification': 3,
  'dsp': 24,
  'json': 15,
  'stdin': 103}}

In [34]:
print('Result (%d ms.) ' % (res['timing']['dsp'] + res['timing']['classification']), end='')
for label in labels:
    score = res['result']['classification'][label]
    print('%s: %.2f\t' % (label, score), end='')
    print('', flush=True)

Result (27 ms.) background-noise: 1.00	
edge-impulse: 0.00	
red-pitaya: 0.00	


In [29]:
#Show the classification key that has the highest confidence index
best_label = max(res['result']['classification'], key=lambda key: res['result']['classification'][key])
confidence = res['result']['classification'][best_label]
print(best_label,"{:.2%}".format(confidence))

background-noise 100.00%
